In [3]:
#Data Preprocessing
#Data downloaded from 
#https://www.airkorea.or.kr/web/last_amb_hour_data?pMENU_NO=123

In [4]:
f = open("./2018_2019_2020.csv", 'r')
f = f.read()
f = f.split("\n")

In [5]:
print(len(f))

11947304


In [6]:
print(f[0],"\n",f[1])

﻿지역,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소 
 서울 중구,111121,중구,2018010101,0.004,0.5,0.02,0.02,34,19,서울 중구 덕수궁길 15


In [7]:
print(f[1].split(","))

['서울 중구', '111121', '중구', '2018010101', '0.004', '0.5', '0.02', '0.02', '34', '19', '서울 중구 덕수궁길 15']


In [8]:
# let's collect pm2.5 (초미세먼지(㎍/㎥)) at 중구
junggu  = []
count = 0
for i in range(1, len(f)):
    line = f[i]
    try:
        if line.split(",")[-1] == '서울 중구 덕수궁길 15':
            try:
                pollutant = line.split(",")[-7:-1]
                val = [ float(val) for val in pollutant ]
                junggu.append(val)
            except:
                # when there is no data copy from previous data point
                junggu.append(junggu[-1])
                count = count + 1
                pass
    except:
        pass

In [28]:
print(count, len(junggu))

1817 24096


In [29]:
# We will predict concentration of pm2.5 based on pm2.5 of previous 24 hours of historical data.
print(junggu[0])

[0.004, 0.5, 0.02, 0.02, 34.0, 19.0]


In [37]:
#normalization by channels
import numpy as np
junggu = np.asarray(junggu)
print(np.shape(junggu))

(24096, 6)


In [38]:
min_list = [ min(junggu[:,i]) for i in range(len(junggu[0,:]))]
max_list = [ max(junggu[:,i]) for i in range(len(junggu[0,:]))]
print(min_list, max_list)

[0.001, 0.2, 0.001, 0.004, 3.0, 1.0] [0.018, 2.1, 0.178, 0.1, 296.0, 149.0]


In [39]:
for i in range(len(junggu)):
    for j in range(len(junggu[0,:])):
        junggu[i,j] = (junggu[i,j] - min_list[j])/ (max_list[j] - min_list[j])

In [41]:
print(junggu[0:3])

[[0.17647059 0.15789474 0.10734463 0.16666667 0.10580205 0.12162162]
 [0.17647059 0.10526316 0.1299435  0.125      0.08191126 0.08783784]
 [0.17647059 0.10526316 0.0960452  0.1875     0.07849829 0.08783784]]


In [42]:
#let's create input(x) and output(y)

In [43]:
x =[]; y =[]
for i in range(24, len(junggu)):
    x.append(junggu[i-24: i])
    y.append([junggu[i][-1]])

In [44]:
len(x), len(y)

(24072, 24072)

In [45]:
#shuffle data
idx = [i for i in range(len(x))] 

In [46]:
import random

In [47]:
random.shuffle(idx)

In [48]:
x_shuffle=[]
y_shuffle=[]
for i in idx:
    x_shuffle.append(x[i])
    y_shuffle.append(y[i])
x,y = x_shuffle, y_shuffle

In [49]:
#change input and output as numpy array
import numpy as np
x = np.asarray(x) 
y = np.asarray(y)

In [50]:
print(np.shape(x), np.shape(y))

(24072, 24, 6) (24072, 1)


In [51]:
# training set: 80% test set: 20%
x_train, y_train = x[0:int(len(x)*0.8)], y[0:int(len(x)*0.8)]
x_test, y_test = x[int(len(x)*0.8):len(x)], y[int(len(x)*0.8): len(x)]

In [52]:
print(np.shape(x_train), np.shape(y_train), np.shape(x_test), np.shape(y_test))

(19257, 24, 6) (19257, 1) (4815, 24, 6) (4815, 1)


In [53]:
# Save it as npy files
np.save("train_x_rnn.npy", x_train)
np.save("train_y_rnn.npy", y_train)
np.save("test_x_rnn.npy", x_test)
np.save("test_y_rnn.npy", y_test)